In [1]:
import sk8s

In [3]:
image = sk8s.docker_build(image_name="ngs-1",
                          conda=["bwa", "gatk4", "samtools", "google-cloud-sdk"],
                          channels=["conda-forge", "bioconda"],
                          pip=["numpy", "scipy", "matplotlib", "pandas"])
image

'gcr.io/jared-genome-analysis/ngs-1'

In [6]:
# Create a volume to store the reference
reference_volume = sk8s.create_volume("100Gi", name="reference_volume", dryrun=True)
print(reference_volume)

apiVersion: v1
kind: PersistentVolumeClaim
metadata:
  name: reference_volume
spec:
  accessModes: ['ReadWriteMany']
  storageClassName: standard-rwx
  resources:
   requests:
    storage: 100Gi


In [ ]:
def populate_reference_volume(volume):
    import subprocess

    def run_silent(cmd):
        return subprocess.run(cmd, check=True, shell=True,
                              stdout=subprocess.PIPE,
                              stderr=subprocess.STDOUT)

    result = run_silent(f"gsutil -m rsync -n -r gs://genomics-public-data/references/hg38/v0/ /mnt/{reference_volume}/hg38/")
    return result.stdout

result = sk8s.run(populate_reference_volume, reference_volume,
                  volumes=[reference_volume], asynchro=False)

In [ ]:
def align(fq1, fq2, output_bam):
    import subprocess
    subprocess.run(f"gsutil -m cp {fq1} ./fq1.fq", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"gsutil -m cp {fq2} ./fq2.fq", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"bwa mem {reference} fq1.fq fq2.fq | samtools sort > out.bam", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    subprocess.run(f"gsutil -m cp out.bam {output_bam}", check=True, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)